In [36]:
import os
import xarray
import re
import pandas as pd
import json
import tempfile
import boto3
import rasterio
from datetime import datetime
from dateutil.relativedelta import relativedelta

# %env AWS_PROFILE='veda-smce-mfa'

In [37]:
session = boto3.Session(profile_name='veda-smce-mfa')
s3_client = session.client('s3')

bucket_name = 'ghgc-data-store-dev'
new_cog_folder = 'updated_with_nodata_values'

In [38]:
def get_all_s3_keys(bucket):
    """Get a list of all keys in an S3 bucket."""
    keys = []

    kwargs = {"Bucket": bucket, "Prefix": "ecco_darwin/"}
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp["Contents"]:
            if obj["Key"].endswith(".tif"):
                keys.append(obj["Key"])

        try:
            kwargs["ContinuationToken"] = resp["NextContinuationToken"]
        except KeyError:
            break

    return keys

In [39]:
keys = get_all_s3_keys(bucket_name)

In [40]:
keys[:10]

['ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200001.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200002.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200003.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200004.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200005.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200006.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200007.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200008.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200009.tif',
 'ODIAC_geotiffs_COGs/odiac2022_1km_excl_intl_200010.tif']

In [41]:
raster_io_session = rasterio.env.Env(profile_name='veda-smce-mfa')

In [42]:
with raster_io_session:
    file = rasterio.open(f's3://{bucket_name}/{keys[0]}')